In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"erixyz","key":"b4dcedcfd61e44926b9e0fc44e50a181"}'}

In [ ]:
#important stuff. always add
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#download the dataset
!kaggle datasets download -d pratik1235/covid19-csea

  0% 0.00/397k [00:00<?, ?B/s]
100% 397k/397k [00:00<00:00, 106MB/s]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

In [ ]:
# importing data, extracting specific csv from zip
zip = zipfile.ZipFile('/content/covid19-csea.zip')
df = pd.read_csv(zip.open('COVID19_line_list_data.csv'))
print(df.shape)
df[:2]

(1085, 27)


,id,case_in_country,reporting date,Unnamed: 3,summary,location,country,gender,age,symptom_onset,...,recovered,symptom,source,link,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,1,NaN,1/20/2020,NaN,First confirmed imported COVID-19 pneumonia pa...,"Shenzhen, Guangdong",China,male,66.0,01/03/20,...,0,NaN,Shenzhen Municipal Health Commission,http://wjw.sz.gov.cn/wzx/202001/t20200120_1898...,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,1/20/2020,NaN,First confirmed imported COVID-19 pneumonia pa...,Shanghai,China,female,56.0,1/15/2020,...,0,NaN,Official Weibo of Shanghai Municipal Health Co...,https://www.weibo.com/2372649470/IqogQhgfa?fro...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# dropping unnecessary data
newdf = df.drop(columns = ['Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
                           'Unnamed: 25', 'Unnamed: 26', 'link', 'Unnamed: 3', 'summary',
                           'from Wuhan', 'visiting Wuhan', 'source', 'location', 'case_in_country',
                           'If_onset_approximated'])
print(newdf.shape)
newdf[:2]

(1085, 12)


,id,reporting date,country,gender,age,symptom_onset,hosp_visit_date,exposure_start,exposure_end,death,recovered,symptom
0,1,1/20/2020,China,male,66.0,01/03/20,01/11/20,12/29/2019,01/04/20,0,0,NaN
1,2,1/20/2020,China,female,56.0,1/15/2020,1/15/2020,NaN,01/12/20,0,0,NaN


In [ ]:
# counting missing values
newdf.isna().sum()

# i noticed death and recovered columns have dates

#newdf.recovered.value_counts()
#newdf.death.value_counts()

id                   0
reporting date       1
country              0
gender             183
age                242
symptom_onset      522
hosp_visit_date    578
exposure_start     957
exposure_end       744
death                0
recovered            0
symptom            815
dtype: int64

In [ ]:
death = newdf[(newdf.death != '0') & (newdf.death != '1')]
recovered = newdf[(newdf.recovered != '0') & (newdf.recovered != '1')]

In [ ]:
# cleaning death dataframe

death = death.drop(columns = ['exposure_start', 'exposure_end', 'recovered',
                              'hosp_visit_date']) # unnecessary cols
death = death[death.country != 'Iran'] # rows with too many missing vals
death = death.drop(labels = 389, axis = 0)
death = death.reset_index()

# changing to datetime
death['reporting date'] = death['reporting date'].apply(pd.to_datetime)
death['symptom_onset'] = death['symptom_onset'].apply(pd.to_datetime)
death['death'] = death['death'].apply(pd.to_datetime)

# days until death
start = []
date1 = death['death'] - death['symptom_onset']
date2 = death['death'] - death['reporting date']
for i in range(len(date1)):
  if pd.isna(date1[i]):
    start.append(date2[i])
  else:
    start.append(date1[i])

death['days'] = start
death.days = (death.days / np.timedelta64(1, 'D')).astype(int)

# dropping odd values
death = death[death.days > 0]

In [ ]:
# cleaning recovered dataframe

recovered = recovered.drop(columns = ['death', 'exposure_start',
                                      'exposure_end', 'hosp_visit_date'])
recovered = recovered.dropna(subset = ['gender', 'age'])

# changing to datetime
recovered['reporting date'] = recovered['reporting date'].apply(pd.to_datetime)
recovered['symptom_onset'] = recovered['symptom_onset'].apply(pd.to_datetime)
recovered['recovered'] = recovered['recovered'].apply(pd.to_datetime)
recovered = recovered.reset_index()
# days until recovery
start = []
date1 = recovered['recovered'] - recovered['symptom_onset']
date2 = recovered['recovered'] - recovered['reporting date']

for i in range(len(date1)):
  if pd.isna(date1[i]):
    start.append(date2[i])
  else:
    start.append(date1[i])

recovered['days'] = start
recovered.days = (recovered.days / np.timedelta64(1, 'D')).astype(int)

# dropping odd values
recovered = recovered[recovered.days > 0]

In [ ]:
recovered = recovered.drop(columns = ['index', 'reporting date', 
                                      'symptom_onset', 'recovered'])
recovered['status'] = 1

In [ ]:
death = death.drop(columns = ['index', 'reporting date', 
                                      'symptom_onset', 'death'])
death['status'] = 0

In [ ]:
alldata = [death, recovered]
data = pd.concat(alldata, ignore_index = True)

In [ ]:
age_group = []
for i in data.age:
  if i <= 20:
    age_group.append('adolescent')  
  elif i > 20 and i <= 35:
    age_group.append('young adult')
  elif i > 35 and i <= 60:
    age_group.append('middle aged')
  else:
    age_group.append('senior')
age_group

data['age_group'] = age_group
data.age_group.value_counts()

middle aged    71
young adult    35
senior         25
adolescent      9
Name: age_group, dtype: int64

In [ ]:
# download data
data.to_csv(r'/content/covid_surv.csv')